In [ ]:
# RTX 4090

In [1]:
!pip install pandas
!pip install matplotlib
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 32.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 36.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 36.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 41.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 128.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgra

In [2]:
!pip install datasets evaluate tensorboard transformers accelerate kagglehub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 6.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 6.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 42.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 98.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 148.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [1]:
import torch
import pandas as pd
import numpy as np
import re
from matplotlib import pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification

from datasets import Dataset
from datasets import concatenate_datasets
from datasets import DatasetDict
from datasets import ClassLabel

import kagglehub
import os

In [ ]:
os.environ["KAGGLE_USERNAME"] = "USERNAME"
os.environ["KAGGLE_KEY"] = "KAGGLE_KEY"

In [2]:
# Set the Seed for Reproducible Results
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [3]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [7]:
# Download the latest version.
dataset_path = kagglehub.dataset_download("anonymous/wolbanking77v1")
dataset_path

100%|██████████| 1.02M/1.02M [00:00<00:00, 1.29MB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/anonymous/wolbanking77v1/versions/4'

In [4]:
df_train = pd.read_csv(
    "/root/.cache/kagglehub/datasets/anonymous/wolbanking77v1/versions/4/wolbanking77v1/5k_split/train/train.csv")

df_test = pd.read_csv(
    "/root/.cache/kagglehub/datasets/anonymous/wolbanking77v1/versions/4/wolbanking77v1/5k_split/test/test.csv")

df_train = df_train[['input_wo', 'label']]
df_test = df_test[['input_wo', 'label']]

#removal of punctuation marks
df_train['input_wo_clean'] = df_train['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')
df_test['input_wo_clean'] = df_test['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')

# Apply lowercase
df_train['input_wo_clean'] = df_train['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_test['input_wo_clean'] = df_test['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))

train_ds = Dataset.from_pandas(df_train[['input_wo_clean', 'label']], split="train")
test_ds = Dataset.from_pandas(df_test[['input_wo_clean', 'label']], split="test")

raw_dataset = DatasetDict({"train": train_ds, "test": test_ds})
# raw_dataset = raw_dataset.remove_columns("__index_level_0__")
raw_dataset =  raw_dataset.rename_column("input_wo_clean", "text") # to match Trainer
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

In [5]:
# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, return_tensors="pt")

In [6]:
from transformers import AutoTokenizer

# Model id to load the tokenizer
model_id = "Davlan/afro-xlmr-large"
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [7]:
tokenized_dataset = raw_dataset.map(tokenize, batched=True,remove_columns=["text"])

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
labels = []
for label in tokenized_dataset["train"]:
    labels.append(label['label'])
labels = list(set(labels))
len(labels)

77

In [10]:
new_features = tokenized_dataset['train'].features.copy()
new_features['label'] = ClassLabel(names=list(set(labels)))
tokenized_dataset['train'] = tokenized_dataset['train'].cast(new_features)
new_features = tokenized_dataset['test'].features.copy()
new_features['label'] = ClassLabel(names=list(set(labels)))
tokenized_dataset['test'] = tokenized_dataset['test'].cast(new_features)

Casting the dataset:   0%|          | 0/4000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
# Prepare model labels - useful for inference
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [12]:
from transformers import AutoModelForSequenceClassification

# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
import evaluate
import numpy as np

# Metric Id
metric = evaluate.load("f1")

# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [15]:
#del trainer
#torch.cuda.empty_cache()

In [14]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments

# Id for remote repository
repository_id = "afro-xlmr-large-banking77-wolof"

# Define training args
training_args = TrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=4, # 8 16
    per_device_eval_batch_size=8, # 8
    learning_rate=2e-05,
	num_train_epochs=20, # 5
    warmup_ratio=0.1,
    weight_decay=0.01,
		# PyTorch 2.0 specifics 
    # bf16=True, # bfloat16 training 
	# torch_compile=True, # optimizations
    optim="adamw_torch_fused", # improved optimizer 
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=200,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
    remove_unused_columns=False

)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [15]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,4.284900,4.126330,0.032902
2,3.046400,2.595688,0.269729
3,1.848200,1.583199,0.530885
4,1.176200,1.227834,0.623857
5,0.909500,1.063811,0.703280
6,0.569100,1.072900,0.693353
7,0.456000,1.056331,0.727728
8,0.258700,1.085393,0.742735
9,0.169400,1.110687,0.746736
10,0.107600,1.162756,0.761136


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



TrainOutput(global_step=20000, training_loss=0.7076851778453216, metrics={'train_runtime': 5058.0352, 'train_samples_per_second': 15.816, 'train_steps_per_second': 3.954, 'total_flos': 7.457340186624e+16, 'train_loss': 0.7076851778453216, 'epoch': 20.0})

In [16]:
trainer.evaluate(tokenized_dataset["test"])

{'eval_loss': 1.4837961196899414,
 'eval_f1': 0.7810571898837346,
 'eval_runtime': 11.8408,
 'eval_samples_per_second': 84.454,
 'eval_steps_per_second': 10.557,
 'epoch': 20.0}

In [17]:
# Save processor and create model card
tokenizer.save_pretrained(repository_id)

('afro-xlmr-large-banking77-wolof/tokenizer_config.json',
 'afro-xlmr-large-banking77-wolof/special_tokens_map.json',
 'afro-xlmr-large-banking77-wolof/tokenizer.json')

In [18]:
model_id = "/workspace/NLP_TASKS/afro-xlmr-large-banking77-wolof/checkpoint-20000"
tokenizer = AutoTokenizer.from_pretrained("/workspace/NLP_TASKS/afro-xlmr-large-banking77-wolof")
model = AutoModelForSequenceClassification.from_pretrained(model_id, use_safetensors=True).to("cuda")

In [19]:
def predict(batch, model):
    #print(batch)
    input_ids = torch.tensor(batch['input_ids']).to(device).unsqueeze(0)
    # token_type_ids = torch.tensor(batch['token_type_ids']).to(device).unsqueeze(0)
    attention_mask = torch.tensor(batch['attention_mask']).to(device).unsqueeze(0)
    with torch.no_grad():
        logits = model(input_ids=input_ids, 
                       #token_type_ids=token_type_ids, 
                       attention_mask=attention_mask).logits
        predicted = torch.argmax(logits, dim=-1)[0]
    return predicted

In [20]:
model.to("cuda")
predictions = [predict(batch, model) for batch in tokenized_dataset["test"]]

In [21]:
predictions[0]

tensor(0, device='cuda:0')

In [22]:
y_pred = []
for y in predictions:
    y_pred.append(labels[y])
y_pred

['topping up by card',
 'Refund not showing up',
 'transfer fee charged',
 'disposable card limits',
 'verify source of funds',
 'card delivery estimate',
 'exchange charge',
 'failed transfer',
 'declined transfer',
 'verify source of funds',
 'order physical card',
 'extra charge on statement',
 'declined card payment',
 'top up by cash or cheque',
 'declined transfer',
 'cash withdrawal not recognised',
 'verify my identity',
 'top up reverted',
 'card payment wrong exchange rate',
 'terminate account',
 'direct debit payment not recognised',
 'wrong amount of cash received',
 'top up reverted',
 'reverted card payment?',
 'card payment wrong exchange rate',
 'why verify identity',
 'transfer timing',
 'cancel transfer',
 'request refund',
 'fiat currency support',
 'why verify identity',
 'top up failed',
 'virtual card not working',
 'declined cash withdrawal',
 'pending cash withdrawal',
 'fiat currency support',
 'country support',
 'card about to expire',
 'transfer not receive

In [23]:
y_test = []
for y in tokenized_dataset["test"]['label']:
    y_test.append(labels[y])
y_test

['topping up by card',
 'Refund not showing up',
 'transfer fee charged',
 'get disposable virtual card',
 'card arrival',
 'card delivery estimate',
 'exchange charge',
 'card delivery estimate',
 'transaction charged twice',
 'verify source of funds',
 'order physical card',
 'extra charge on statement',
 'declined card payment',
 'apple pay or google pay',
 'top up reverted',
 'cash withdrawal not recognised',
 'verify my identity',
 'wrong amount of cash received',
 'card payment wrong exchange rate',
 'terminate account',
 'direct debit payment not recognised',
 'wrong amount of cash received',
 'balance not updated after cheque or cash deposit',
 'reverted card payment?',
 'card payment wrong exchange rate',
 'why verify identity',
 'transfer timing',
 'cancel transfer',
 'request refund',
 'verify my identity',
 'verify my identity',
 'top up reverted',
 'virtual card not working',
 'wrong amount of cash received',
 'pending cash withdrawal',
 'fiat currency support',
 'country 

In [24]:
# from sklearn import metrics
# y_test = test_df['label']
print(metrics.classification_report(y_test, y_pred))

                                                  precision    recall  f1-score   support

                           Refund not showing up       0.93      0.76      0.84        17
                                activate my card       0.71      0.75      0.73        16
                                       age limit       1.00      1.00      1.00        10
                         apple pay or google pay       1.00      0.79      0.88        14
                                     atm support       0.90      1.00      0.95         9
                                automatic top up       0.82      1.00      0.90        14
         balance not updated after bank transfer       0.47      0.53      0.50        17
balance not updated after cheque or cash deposit       0.78      0.82      0.80        17
                         beneficiary not allowed       0.60      0.40      0.48        15
                                 cancel transfer       0.64      1.00      0.78        14
         

In [25]:
metrics.f1_score(y_test, y_pred, average='weighted')

0.7782164644362767

In [26]:
metrics.accuracy_score(y_test, y_pred)

0.781

In [27]:
metrics.precision_score(y_test, y_pred, average='weighted')

0.7906492666990732

In [28]:
metrics.recall_score(y_test, y_pred, average='weighted')

0.781

In [29]:
def compute_metrics(y_true, y_pred):
    print("f1_score:", metrics.f1_score(y_true, y_pred, average='weighted'))
    print("precision_score:", metrics.precision_score(y_true, y_pred, average='weighted'))
    print("recall_score:", metrics.recall_score(y_true, y_pred, average='weighted'))

In [30]:
compute_metrics(y_test, y_pred)

f1_score: 0.7782164644362767
precision_score: 0.7906492666990732
recall_score: 0.781
